In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# explore_test_data.py
import os

def explore_test_data():
    test_path = "/content/drive/MyDrive/Research Rangers/dataset_final_v1"  # or wherever your test data is
    if not os.path.exists(test_path):
        print("Test data not found! Check the path.")
        return

    print("=== EXPLORING TEST DATA STRUCTURE ===")

    # List all files and folders in test directory
    for item in os.listdir(test_path):
        item_path = os.path.join(test_path, item)
        if os.path.isdir(item_path):
            print(f"📁 Folder: {item}")
            # Show subfolders/files
            subitems = os.listdir(item_path)[:5]  # First 5 items
            for subitem in subitems:
                subitem_path = os.path.join(item_path, subitem)
                if os.path.isdir(subitem_path):
                    print(f"   📁 {subitem}/")
                else:
                    print(f"   📄 {subitem}")
        else:
            print(f"📄 File: {item}")

    # Count total images
    image_count = 0
    for root, dirs, files in os.walk(test_path):
        for file in files:
            if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                image_count += 1

    print(f"\nTotal images found: {image_count}")

if __name__ == "__main__":
    explore_test_data()

=== EXPLORING TEST DATA STRUCTURE ===
📁 Folder: val
   📄 labels.cache
   📁 images/
   📁 labels/
📁 Folder: train
   📄 labels.cache
   📁 labels/
   📁 images/

Total images found: 758


In [7]:
# src/proper_predict.py
import torch
import torch.nn as nn
import cv2
import numpy as np
import os
import zipfile
import pandas as pd
from torchvision import transforms
import torch.nn.functional as F

class SimpleSegmentationModel(nn.Module):
    def __init__(self):
        super(SimpleSegmentationModel, self).__init__()
        # Simple encoder-decoder architecture
        self.encoder = nn.Sequential(
            # Input: 3 x 256 x 256
            nn.Conv2d(3, 64, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 64 x 128 x 128

            nn.Conv2d(64, 128, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 128 x 64 x 64

            nn.Conv2d(128, 256, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 256 x 32 x 32
        )

        self.decoder = nn.Sequential(
            # 256 x 32 x 32
            nn.Conv2d(256, 128, 3, padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2, mode='bilinear'),  # 128 x 64 x 64

            nn.Conv2d(128, 64, 3, padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2, mode='bilinear'),  # 64 x 128 x 128

            nn.Conv2d(64, 32, 3, padding=1),
            nn.ReLU(),
            nn.Upsample(scale_factor=2, mode='bilinear'),  # 32 x 256 x 256

            nn.Conv2d(32, 1, 1),
            nn.Sigmoid()  # Output probabilities 0-1
        )

    def forward(self, x):
        features = self.encoder(x)
        return self.decoder(features)

def predict_test_set():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    # Initialize simple model
    model = SimpleSegmentationModel().to(device)
    model.eval()
    print("Model initialized (random weights)")

    # Find all test images
    test_images = []
    test_dirs = ["/content/drive/MyDrive/Research Rangers/dataset_final_v1"]  # Adjust this path if needed

    for test_dir in test_dirs:
        if os.path.exists(test_dir):
            for root, dirs, files in os.walk(test_dir):
                for file in files:
                    if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                        test_images.append(os.path.join(root, file))

    print(f"Found {len(test_images)} test images")

    if len(test_images) == 0:
        print("No test images found! Check your test data path.")
        return

    # Create submissions directory
    os.makedirs('submissions', exist_ok=True)
    temp_mask_dir = 'submissions/temp_masks'
    os.makedirs(temp_mask_dir, exist_ok=True)

    submission_data = []

    # Simple preprocessing
    transform = transforms.Compose([
        transforms.ToTensor(),
    ])

    with torch.no_grad():
        for i, image_path in enumerate(test_images):
            if i % 100 == 0:
                print(f"Processing {i}/{len(test_images)}...")

            # Load image
            image = cv2.imread(image_path)
            if image is None:
                print(f"Could not load {image_path}, skipping...")
                continue

            original_size = image.shape[:2]
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Resize for model
            input_size = 256
            image_resized = cv2.resize(image_rgb, (input_size, input_size))

            # Preprocess
            image_tensor = transform(image_resized).unsqueeze(0).to(device)

            # Predict (this will be random but consistent)
            pred_mask = model(image_tensor)
            pred_mask = pred_mask.squeeze().cpu().numpy()

            # Resize back to original dimensions
            pred_mask = cv2.resize(pred_mask, (original_size[1], original_size[0]))

            # Calculate global probability (max value in mask)
            global_prob = float(np.max(pred_mask))
            global_label = 1 if global_prob > 0.5 else 0

            # Prepare submission data
            image_id = os.path.basename(image_path)
            submission_data.append({
                'image_id': image_id,
                'prob': global_prob,
                'label': global_label,
                'threshold': 0.5,
                'loc_threshold': 0.5
            })

            # Save mask as .npz
            npz_filename = os.path.splitext(image_id)[0] + '.npz'
            npz_path = os.path.join(temp_mask_dir, npz_filename)
            np.savez_compressed(npz_path, pred_mask.astype(np.float16))

    # Create ZIP file of masks
    zip_path = 'submissions/teamname_localization_masks.zip'
    with zipfile.ZipFile(zip_path, 'w') as zipf:
        for file in os.listdir(temp_mask_dir):
            file_path = os.path.join(temp_mask_dir, file)
            zipf.write(file_path, file)

    # Create scores.csv
    df = pd.DataFrame(submission_data)
    csv_path = 'submissions/scores.csv'
    df.to_csv(csv_path, index=False)

    # Cleanup
    import shutil
    shutil.rmtree(temp_mask_dir)

    print(f"✅ Submission created!")
    print(f"   - Masks: {zip_path}")
    print(f"   - Scores: {csv_path}")
    print(f"   - Total images processed: {len(test_images)}")
    print(f"   - Note: Using random weights - for competition, train on TGIF data first")

if __name__ == "__main__":
    predict_test_set()

Using device: cuda
Model initialized (random weights)
Found 868 test images
Processing 0/868...
Processing 100/868...
Processing 200/868...
Processing 300/868...
Processing 400/868...
Processing 500/868...
Processing 600/868...
Processing 700/868...
Processing 800/868...
✅ Submission created!
   - Masks: submissions/teamname_localization_masks.zip
   - Scores: submissions/scores.csv
   - Total images processed: 868
   - Note: Using random weights - for competition, train on TGIF data first


In [9]:
# final_verification.py
import os
import zipfile
import pandas as pd
import numpy as np

def final_verification():
    print("=== FINAL SUBMISSION VERIFICATION ===")

    zip_path = 'submissions/Research_Rangers_localization_masks.zip'
    csv_path = 'submissions/scores.csv'

    # Check file existence
    if not os.path.exists(zip_path):
        print("❌ ZIP file missing!")
        return False
    if not os.path.exists(csv_path):
        print("❌ CSV file missing!")
        return False

    print("✅ Both files exist")

    # Verify CSV structure
    df = pd.read_csv(csv_path)
    required_columns = ['image_id', 'prob', 'label', 'threshold', 'loc_threshold']

    if list(df.columns) != required_columns:
        print(f"❌ CSV columns incorrect. Expected: {required_columns}")
        print(f"    Got: {list(df.columns)}")
        return False

    print("✅ CSV columns correct")

    # Verify data ranges
    if (df['prob'] < 0).any() or (df['prob'] > 1).any():
        print("❌ Probabilities outside 0-1 range")
        return False

    if not df['label'].isin([0, 1]).all():
        print("❌ Labels not binary (0 or 1)")
        return False

    print("✅ Data ranges valid")

    # Verify ZIP contents
    with zipfile.ZipFile(zip_path, 'r') as zipf:
        zip_files = zipf.namelist()

        # Check if all CSV images have corresponding NPZ files
        csv_images = set(df['image_id'].str.replace(r'\.(jpg|jpeg|png)$', '.npz', regex=True))
        zip_npz_files = set([f for f in zip_files if f.endswith('.npz')])

        if len(csv_images) != len(zip_npz_files):
            print("❌ Mismatch between CSV entries and NPZ files")
            print(f"    CSV has {len(csv_images)} images, ZIP has {len(zip_npz_files)} NPZ files")
            return False

        # Check one NPZ file structure
        if zip_npz_files:
            sample_npz = list(zip_npz_files)[0]
            with zipf.open(sample_npz) as f:
                npz_data = np.load(f)
                mask = npz_data['arr_0']  # Default name from np.savez_compressed

                if mask.dtype != np.float16:
                    print(f"❌ Mask dtype is {mask.dtype}, should be float16")
                    return False

                if np.min(mask) < 0 or np.max(mask) > 1:
                    print("❌ Mask values outside 0-1 range")
                    return False

    print("✅ ZIP file structure correct")
    print("🎉 SUBMISSION READY! You can submit these files:")
    print(f"   - {zip_path}")
    print(f"   - {csv_path}")
    return True

if __name__ == "__main__":
    final_verification()

=== FINAL SUBMISSION VERIFICATION ===
✅ Both files exist
✅ CSV columns correct
✅ Data ranges valid
✅ ZIP file structure correct
🎉 SUBMISSION READY! You can submit these files:
   - submissions/Research_Rangers_localization_masks.zip
   - submissions/scores.csv
